In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from ECOv002_calval_tables import load_calval_table
from FLiESANN import load_ECOv002_static_tower_FLiESANN_inputs
from FLiESANN import process_FLiESANN_table

[2025-10-23 14:38:14 INFO] SRTM working directory: ~/data/NASADEM
[2025-10-23 14:38:14 INFO] SRTM download directory: ~/data/NASADEM
[2025-10-23 14:38:14 INFO] SRTM download directory: ~/data/NASADEM


In [3]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'FLiESANN')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-FLiESANN-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-FLiESANN-outputs.csv")

In [4]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf["elevation_km"] = model_inputs_gdf["Elev"] / 1000.0
model_inputs_gdf.head()

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,emissivity,insitu_LE_Wm2,insitu_H_Wm2,insitu_Rn_Wm2,insitu_G_Wm2,insitu_SWin_Wm2,insitu_Rn_daylight_Wm2,insitu_LE_daylight_Wm2,insitu_ET_daylight_kg,elevation_km
0,0,US-NC3,ENF,Cfa,270.34520,78.53355,392.85184,307.02197,487.383423,118.91628,...,0.948,331.723100,69.623470,449.65123,14.831077,596.8641,268.829847,205.089552,3.521847,0.005
1,1,US-Mi3,CVM,Dfb,232.14160,229.20093,640.11847,375.08930,106.825577,167.91946,...,0.952,276.182074,228.756306,667.81900,38.638200,NaN,346.382557,152.046364,3.361398,0.270
2,2,US-Mi3,CVM,Dfb,356.35574,335.23154,625.66170,284.68625,NaN,132.93634,...,0.972,336.849808,169.322709,603.26000,26.775450,NaN,311.373843,181.941076,4.018909,0.270
3,3,US-Mi3,CVM,Dfb,332.93840,326.68680,624.25433,251.41449,178.827545,141.13242,...,0.974,276.264202,231.754374,551.03800,13.319750,NaN,298.014172,153.111127,3.377459,0.270
4,4,US-Mi3,CVM,Dfb,286.85403,237.21654,511.08218,228.52017,154.791626,114.80941,...,0.960,192.205091,155.974781,512.27100,6.896745,NaN,300.483101,114.280419,2.519399,0.270


In [5]:
static_inputs_df = load_ECOv002_static_tower_FLiESANN_inputs()
static_inputs_df

,ID,name,KG_climate,COT,AOT,Ca,wind_speed_mps,vapor_gccm,ozone_cm,geometry
0,US-NC3,NC_Clearcut#3,3,0,0,400,0,0,0.3,POINT (-76.656 35.799)
1,PE-QFR,Quistococha Forest Reserve,1,0,0,400,0,0,0.3,POINT (-73.319 -3.8344)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,4,0,0,400,0,0,0.3,POINT (-80.637 41.8222)
3,US-NC4,NC_AlligatorRiver,3,0,0,400,0,0,0.3,POINT (-75.9038 35.7879)
4,CA-DB2,Delta Burns Bog 2,3,0,0,400,0,0,0.3,POINT (-122.9951 49.119)
...,...,...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",2,0,0,400,0,0,0.3,POINT (-103.0293 40.4619)
117,US-xWD,NEON Woodworth (WOOD),4,0,0,400,0,0,0.3,POINT (-99.2414 47.1282)
118,US-CS4,Central Sands Irrigated Agricultural Field,4,0,0,400,0,0,0.3,POINT (-89.5475 44.1597)
119,US-xAE,NEON Klemme Range Research Station (OAES),3,0,0,400,0,0,0.3,POINT (-99.0588 35.4106)


In [6]:
# merge static inputs with model inputs, ignoring duplicate columns from static_inputs_df
cols_to_use = [col for col in static_inputs_df.columns if col not in model_inputs_gdf.columns or col == 'ID']
model_inputs_gdf = model_inputs_gdf.merge(static_inputs_df[cols_to_use], on="ID", how="left")
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,insitu_ET_daylight_kg,elevation_km,name,KG_climate,COT,AOT,Ca,wind_speed_mps,vapor_gccm,ozone_cm
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,3.521847,0.005,NC_Clearcut#3,3,0,0,400,0,0,0.3
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,3.361398,0.270,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,4,0,0,400,0,0,0.3
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,4.018909,0.270,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,4,0,0,400,0,0,0.3
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,3.377459,0.270,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,4,0,0,400,0,0,0.3
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,2.519399,0.270,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,4,0,0,400,0,0,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,0.094448,0.516,NEON Klemme Range Research Station (OAES),3,0,0,400,0,0,0.3
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,1.284029,0.516,NEON Klemme Range Research Station (OAES),3,0,0,400,0,0,0.3
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,1.461298,0.516,NEON Klemme Range Research Station (OAES),3,0,0,400,0,0,0.3
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,1.500961,0.516,NEON Klemme Range Research Station (OAES),3,0,0,400,0,0,0.3


In [7]:
model_inputs_gdf.columns

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C', 'SWin_Wm2', 'emissivity', 'insitu_LE_Wm2', 'insitu_H_Wm2',
       'insitu_Rn

In [8]:
results = process_FLiESANN_table(model_inputs_gdf)
results

[2025-10-23 14:38:14 INFO] started extracting geometry from FLiES input table
[2025-10-23 14:38:14 INFO] completed extracting geometry from FLiES input table
[2025-10-23 14:38:14 INFO] started extracting time from FLiES input table
[2025-10-23 14:38:14 INFO] completed extracting time from FLiES input table
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,NIR_diffuse_Wm2,PAR_direct_Wm2,NIR_direct_Wm2,atmospheric_transmittance,UV_proportion,PAR_proportion,NIR_proportion,UV_diffuse_fraction,PAR_diffuse_fraction,NIR_diffuse_fraction
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,0.000000,290.492161,370.061337,0.828220,0.061075,0.435837,0.505264,0.428990,0.089973,0.000000
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,0.000000,437.988509,535.473995,0.877727,0.060632,0.432938,0.508693,0.293193,0.038931,0.000000
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,0.000000,441.879613,539.073476,0.879420,0.060702,0.432878,0.508685,0.291155,0.036749,0.000000
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,0.000000,401.138166,500.646551,0.859784,0.059538,0.433182,0.509790,0.305480,0.057060,0.000000
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,4.714088,347.796466,440.019955,0.839109,0.059503,0.434459,0.508510,0.345300,0.084674,0.010600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,8.190334,138.437136,189.167959,0.761245,0.048996,0.430012,0.524395,0.522320,0.144589,0.041500
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,7.003343,153.199717,204.545958,0.773786,0.050539,0.433847,0.518522,0.514120,0.134479,0.033105
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,5.034356,173.970245,229.450109,0.789060,0.051972,0.435507,0.515233,0.497488,0.122254,0.021470
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,0.000000,217.930880,287.005670,0.807671,0.054816,0.434220,0.513771,0.461162,0.101562,0.000000


In [9]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [10]:
results.to_csv(generated_output_table_filename, index=False)